## 1. Imports and Configuration

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
import json
from datetime import datetime
from pathlib import Path

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Model Selection & Tuning
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, train_test_split

# Evaluation Metrics
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve, auc, ConfusionMatrixDisplay
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Gradient Boosting
import lightgbm as lgb

# Display Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("All libraries imported successfully!")

## 2. Data Loading and Preprocessing

In [ ]:
# Load the feature-engineered dataset
df = pd.read_csv(FEATURE_PATH)
print(f"Original dataset shape: {df.shape}")

# Validate presence of required label columns
assert "label1" in df.columns, "label1 column not found in dataset"
assert "label2" in df.columns, "label2 column not found in dataset"

# Filter to retain only attack traffic (exclude benign)
df_attacks = df[df["label1"].str.lower() == "attack"].copy()
print(f"\nAttack traffic records: {df_attacks.shape[0]}")
print(f"Percentage of attack traffic: {(df_attacks.shape[0] / df.shape[0] * 100):.2f}%")

# Analyze attack type distribution
print(f"\n{'='*60}")
print("Attack Type Distribution (Label2)")
print(f"{'='*60}")
print(df_attacks["label2"].value_counts())
print(f"\nNumber of attack classes: {df_attacks['label2'].nunique()}")

# Extract features and create multiclass target variable
# Target: label2 (attack sub-types)
X = df_attacks.drop(columns=["label1", "label2", "label3", "label4", "label_full"], errors='ignore')
y = df_attacks["label2"].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"\nClass distribution:\n{y.value_counts()}")

## 3. Train-Test Split

In [ ]:
RANDOM_STATE = 42

# Perform stratified train-test split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"\nTraining set class distribution:\n{y_train.value_counts()}")

## 4. Multiclass Metrics Computation

In [ ]:
model_defs = {}

# ============================================================================
# LINEAR MODELS
# ============================================================================

# Logistic Regression - Multi-class linear classifier
model_defs["log_reg"] = {
    "estimator": LogisticRegression(max_iter=5000, random_state=RANDOM_STATE, multi_class="multinomial"),
    "param_grid": {
        "penalty": ["l2","elasticnet"],
        "C": [0.1, 1, 10, 100],
        "solver": ["lbfgs", "saga"],
        "class_weight": [None, "balanced"]
    }
}

# ============================================================================
# TREE-BASED MODELS
# ============================================================================

# Random Forest - Ensemble of decision trees
model_defs["rf"] = {
    "estimator": RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
    "param_grid": {
        "n_estimators": [50, 100, 200],
        "max_depth": [10, 20, 30, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt"],
        "class_weight": [None, "balanced"]
    }
}

# LightGBM - Fast gradient boosting classifier
model_defs["lgb"] = {
    "estimator": lgb.LGBMClassifier(
        random_state=RANDOM_STATE, 
        n_jobs=-1, 
        verbose=-1,
        is_unbalance=True
    ),
    "param_grid": {
        "n_estimators": [100, 200, 300],
        "max_depth": [5, 10, 15, -1],
        "learning_rate": [0.01, 0.05, 0.1],
        "num_leaves": [20, 30, 40, 50],
        "min_data_in_leaf": [10, 20, 30],
        "feature_fraction": [0.8, 0.9, 1.0],
        "bagging_fraction": [0.8, 0.9, 1.0]
    }
}

# ============================================================================
# NEURAL NETWORK MODELS
# ============================================================================

# Multi-Layer Perceptron - Neural network classifier for multi-class
model_defs["mlp"] = {
    "estimator": MLPClassifier(random_state=RANDOM_STATE, max_iter=500, early_stopping=True, n_jobs=-1),
    "param_grid": {
        "hidden_layer_sizes": [(64,), (128,), (64, 32), (128, 64)],
        "activation": ["relu", "tanh"],
        "alpha": [0.0001, 0.001, 0.01],
        "learning_rate": ["constant", "adaptive"],
        "batch_size": [32, 64]
    }
}

# ============================================================================
# DISTANCE-BASED & PROBABILISTIC MODELS
# ============================================================================

# K-Nearest Neighbors - Distance-based instance classifier
model_defs["knn"] = {
    "estimator": KNeighborsClassifier(),
    "param_grid": {
        "n_neighbors": [3, 5, 7, 9, 11],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"]
    }
}

# Gaussian Naive Bayes - Probabilistic classifier
model_defs["nb"] = {
    "estimator": GaussianNB(),
    "param_grid": {
        "var_smoothing": np.logspace(-10, -6, 5)
    }
}

# ============================================================================
# MODEL TRAINING CONFIGURATION
# ============================================================================

MODELS_TO_RUN = ["log_reg", "rf", "lgb", "mlp", "knn", "nb"]

print(f"Models to train: {MODELS_TO_RUN}")
print(f"Optimization method: RandomizedSearchCV")
print(f"Cross-validation folds: 3")
print(f"Iterations per model: 10")
print(f"Total models: {len(MODELS_TO_RUN)}")

## 5. Model Definition and Hyperparameter Grids

In [ ]:
model_defs = {}

# ============================================================================
# LINEAR MODELS
# ============================================================================

# Logistic Regression - Fast baseline linear classifier for multiclass
model_defs["log_reg"] = {
    "estimator": LogisticRegression(max_iter=5000, random_state=RANDOM_STATE, multi_class='multinomial'),
    "param_grid": {
        "penalty": ["l2"],
        "C": [0.1, 1, 10, 100],
        "solver": ["lbfgs", "saga"],
        "class_weight": [None, "balanced"]
    }
}

# Stochastic Gradient Descent - Fast linear classifier for multiclass
model_defs["sgd"] = {
    "estimator": SGDClassifier(random_state=RANDOM_STATE, n_jobs=-1, early_stopping=True, loss='log_loss'),
    "param_grid": {
        "loss": ["log_loss", "modified_huber"],
        "penalty": ["l2", "l1", "elasticnet"],
        "alpha": [0.0001, 0.001, 0.01, 0.1],
        "max_iter": [100, 200, 500],
        "class_weight": [None, "balanced"]
    }
}

# ============================================================================
# TREE-BASED MODELS
# ============================================================================

# Decision Tree - Interpretable tree-based classifier
model_defs["dt"] = {
    "estimator": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "param_grid": {
        "criterion": ["gini", "entropy"],
        "max_depth": [10, 20, 30, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2"],
        "class_weight": [None, "balanced"]
    }
}

# Random Forest - Ensemble of decision trees
model_defs["rf"] = {
    "estimator": RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
    "param_grid": {
        "n_estimators": [50, 100, 200],
        "max_depth": [10, 20, 30, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt"],
        "class_weight": [None, "balanced"]
    }
}

# LightGBM - Fast gradient boosting classifier for multiclass
model_defs["lgb"] = {
    "estimator": lgb.LGBMClassifier(
        random_state=RANDOM_STATE, 
        n_jobs=-1, 
        verbose=-1,
        is_unbalance=True,
        num_leaves=31
    ),
    "param_grid": {
        "n_estimators": [100, 200, 300],
        "max_depth": [5, 10, 15, -1],
        "learning_rate": [0.01, 0.05, 0.1],
        "num_leaves": [20, 30, 40],
        "min_data_in_leaf": [10, 20, 30],
        "feature_fraction": [0.8, 0.9, 1.0],
        "bagging_fraction": [0.8, 0.9, 1.0]
    }
}

# ============================================================================
# DISTANCE-BASED & PROBABILISTIC MODELS
# ============================================================================

# K-Nearest Neighbors - Distance-based instance classifier
model_defs["knn"] = {
    "estimator": KNeighborsClassifier(),
    "param_grid": {
        "n_neighbors": [3, 5, 7, 9, 11],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"]
    }
}

# Gaussian Naive Bayes - Probabilistic classifier
model_defs["nb"] = {
    "estimator": GaussianNB(),
    "param_grid": {
        "var_smoothing": np.logspace(-10, -6, 5)
    }
}

# ============================================================================
# MODEL TRAINING CONFIGURATION
# ============================================================================

MODELS_TO_RUN = ["log_reg", "sgd", "dt", "rf", "lgb", "knn", "nb"]

print(f"Models to train: {MODELS_TO_RUN}")
print(f"Optimization method: RandomizedSearchCV")
print(f"Cross-validation folds: 3")
print(f"Iterations per model: 10")
print(f"Scoring metric: F1 (macro-averaged for multiclass)")
print(f"Total models: {len(MODELS_TO_RUN)}")

## 6. Model Training and Evaluation Function

In [ ]:
def train_and_evaluate_model(model_key, model_def, X_train, X_test, y_train, y_test,
                             base_dir, cv_folds=3, n_iter=10, class_labels=None):
    """
    Train a multiclass model using RandomizedSearchCV and evaluate on test set.
    
    Parameters:
    -----------
    model_key : str
        Model identifier
    model_def : dict
        Dictionary containing estimator and parameter grid
    X_train, X_test : array-like
        Training and test feature matrices
    y_train, y_test : array-like
        Training and test labels
    base_dir : str
        Base directory for saving results
    cv_folds : int
        Number of cross-validation folds
    n_iter : int
        Number of RandomizedSearchCV iterations
    class_labels : list
        List of class labels for visualization
        
    Returns:
    --------
    tuple : (results_dict, best_model, predictions, metrics)
    """
    estimator = model_def["estimator"]
    param_grid = model_def["param_grid"]

    MODEL_DIR = os.path.join(base_dir, model_key)
    os.makedirs(MODEL_DIR, exist_ok=True)

    print(f"\n{'='*60}")
    print(f"Training {model_key.upper()}")
    print(f"{'='*60}")

    # Perform hyperparameter tuning using RandomizedSearchCV
    if param_grid:
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
        search = RandomizedSearchCV(
            estimator,
            param_distributions=param_grid,
            n_iter=n_iter,
            scoring="f1_macro",
            cv=cv,
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=1
        )
        search.fit(X_train, y_train)
        best_model = search.best_estimator_
        best_params = search.best_params_
        cv_f1 = search.best_score_
    else:
        best_model = estimator
        best_model.fit(X_train, y_train)
        best_params = {}
        cv_f1 = np.nan

    print(f"Best parameters: {best_params}")
    print(f"Cross-validation F1 score (macro): {cv_f1:.4f}")

    # Generate predictions on test set
    y_pred = best_model.predict(X_test)

    # Compute comprehensive metrics
    metrics = compute_metrics_multiclass(y_test, y_pred)

    # Generate and save confusion matrix visualization
    cm = metrics["confusion_matrix"]
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_labels, yticklabels=class_labels,
                cbar_kws={'label': 'Count'})
    plt.title(f"{model_key.upper()} - Confusion Matrix (Multiclass)")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    cm_path = os.path.join(MODEL_DIR, f"{model_key}_confusion_matrix.png")
    plt.tight_layout()
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Visualizations saved for {model_key}")

    # Return summary metrics
    return {
        "model": model_key,
        "best_params": str(best_params),
        "cv_f1_macro": cv_f1,
        "test_accuracy": metrics["accuracy"],
        "test_precision_macro": metrics["precision_macro"],
        "test_recall_macro": metrics["recall_macro"],
        "test_f1_macro": metrics["f1_macro"],
        "test_f1_weighted": metrics["f1_weighted"]
    }, best_model, y_pred, metrics

## 7. Main Training Loop

In [ ]:
# Extract class labels for visualization
class_labels = sorted(y_train.unique())

# Generate timestamp for unique results directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = os.path.join("multiclass_classification", f"results_{timestamp}")
os.makedirs(RUN_DIR, exist_ok=True)
print(f"Results will be saved to: {RUN_DIR}\n")

results_list = []
models_dict = {}
y_preds_dict = {}
metrics_dict = {}

# Train all models using optimized RandomizedSearchCV
for key in MODELS_TO_RUN:
    result_row, best_model, y_pred, metrics = train_and_evaluate_model(
        model_key=key,
        model_def=model_defs[key],
        X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
        base_dir=RUN_DIR,
        cv_folds=3,
        n_iter=10,
        class_labels=class_labels
    )
    results_list.append(result_row)
    models_dict[key] = best_model
    y_preds_dict[key] = y_pred
    metrics_dict[key] = metrics

print(f"\n{'='*60}")
print("MODEL TRAINING COMPLETED SUCCESSFULLY")
print(f"{'='*60}\n")

# Create metrics summary dataframe
metrics_df = pd.DataFrame(results_list)
metrics_csv = os.path.join(RUN_DIR, "01_metrics_summary_all_models.csv")
metrics_df.to_csv(metrics_csv, index=False)
print(f"Metrics summary saved: {metrics_csv}\n")
display(metrics_df)

# Identify best performing model based on F1-macro
best_model_key = metrics_df.loc[metrics_df["test_f1_macro"].idxmax(), "model"]
best_model_f1 = metrics_df.loc[metrics_df["test_f1_macro"].idxmax(), "test_f1_macro"]
print(f"\n{'='*60}")
print(f"BEST MODEL: {best_model_key.upper()}")
print(f"Test F1-Score (Macro): {best_model_f1:.4f}")
print(f"{'='*60}\n")

## 8. Best Model Detailed Report

In [ ]:
# Extract best model results
best_model_obj = models_dict[best_model_key]
y_pred_best = y_preds_dict[best_model_key]
metrics_best = metrics_dict[best_model_key]

# Save best model to its results directory
best_model_path = os.path.join(RUN_DIR, f"best_multiclass_model_{best_model_key}.pkl")
joblib.dump(best_model_obj, best_model_path)
print(f"Best model saved: {best_model_path}\n")

# Also save to production directory for deployment
best_model_prod_path = os.path.join(MODELS_DIR, f"best_multiclass_attack_classification_model.pkl")
joblib.dump(best_model_obj, best_model_prod_path)
print(f"Best model also saved to production: {best_model_prod_path}\n")

# Generate classification report for best model
class_report = classification_report(y_test, y_pred_best,
                                     target_names=class_labels,
                                     digits=4)

# Create detailed report text
report_text = f"""
{'='*70}
MULTICLASS ATTACK CLASSIFICATION - BEST MODEL REPORT
{'='*70}

Execution Timestamp: {timestamp}
Best Model: {best_model_key.upper()}
Results Location: {best_model_path}
Production Location: {best_model_prod_path}
Attack Classes: {', '.join(class_labels)}
Total Classes: {len(class_labels)}

{'='*70}
MODEL PERFORMANCE METRICS
{'='*70}

Accuracy (overall):         {metrics_best['accuracy']:.4f}
Precision (macro):          {metrics_best['precision_macro']:.4f}
Recall (macro):             {metrics_best['recall_macro']:.4f}
F1-Score (macro):           {metrics_best['f1_macro']:.4f}
F1-Score (weighted):        {metrics_best['f1_weighted']:.4f}

Confusion Matrix Shape: {metrics_best['confusion_matrix'].shape}

{'='*70}
CLASSIFICATION REPORT
{'='*70}

{class_report}

{'='*70}
BEST MODEL HYPERPARAMETERS
{'='*70}

{str(model_defs[best_model_key]['param_grid'])}

{'='*70}
"""

# Save report to file
report_path = os.path.join(RUN_DIR, "02_best_model_report.txt")
with open(report_path, "w") as f:
    f.write(report_text)

print("Best model report generated and saved")
print(report_text)

## 9. Model Comparison Visualizations

In [ ]:
# 1. Performance metrics comparison across models
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle("Multiclass Model Performance Comparison", fontsize=16, fontweight='bold')

metrics_to_plot = ["test_accuracy", "test_precision_macro", "test_recall_macro",
                   "test_f1_macro", "test_f1_weighted", "cv_f1_macro"]
colors = ['#FF6B6B' if model == best_model_key else '#4ECDC4'
          for model in metrics_df["model"]]

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    bars = ax.bar(metrics_df["model"], metrics_df[metric], color=colors, alpha=0.8, edgecolor='black')
    ax.set_ylabel(metric.replace("test_", "").replace("_", " "), fontsize=10)
    ax.set_title(metric.replace("test_", "").replace("_", " ").upper(), fontsize=11, fontweight='bold')
    ax.set_ylim([0, 1.05])
    ax.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
comparison_path = os.path.join(RUN_DIR, "03_models_comparison_metrics.png")
plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
plt.close()
print("Saved: models_comparison_metrics.png")

# 2. F1-score ranking
fig, ax = plt.subplots(figsize=(10, 6))
sorted_df = metrics_df.sort_values("test_f1_macro", ascending=True)
colors_rank = ['#FF6B6B' if model == best_model_key else '#95E1D3'
               for model in sorted_df["model"]]
bars = ax.barh(sorted_df["model"], sorted_df["test_f1_macro"], color=colors_rank, edgecolor='black', alpha=0.85)

ax.set_xlabel("F1-Score (Macro)", fontsize=12)
ax.set_title("Models Ranked by F1-Score (Macro)", fontsize=14, fontweight='bold')
ax.set_xlim([0, 1.05])
ax.grid(axis='x', alpha=0.3)

for idx, bar in enumerate(bars):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f' {width:.4f}', ha='left', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
ranking_path = os.path.join(RUN_DIR, "04_f1_score_ranking.png")
plt.savefig(ranking_path, dpi=300, bbox_inches='tight')
plt.close()
print("Saved: f1_score_ranking.png")

# 3. Metrics heatmap
fig, ax = plt.subplots(figsize=(12, 6))
metrics_for_heatmap = metrics_df[[
    "model", "test_accuracy", "test_precision_macro",
    "test_recall_macro", "test_f1_macro", "test_f1_weighted"
]].set_index("model")
sns.heatmap(metrics_for_heatmap.T, annot=True, fmt='.4f', cmap='RdYlGn',
            cbar_kws={'label': 'Score'}, ax=ax, linewidths=0.5)
ax.set_title("Multiclass Model Metrics Heatmap", fontsize=14, fontweight='bold')
plt.tight_layout()
heatmap_path = os.path.join(RUN_DIR, "05_metrics_heatmap.png")
plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
plt.close()
print("Saved: metrics_heatmap.png")

print("\nAll comparison visualizations generated successfully")

## 10. Execution Summary and Results Verification

In [ ]:
print(f"\n{'='*70}")
print("RESULTS DIRECTORY STRUCTURE")
print(f"{'='*70}\n")

# List all generated files in results directory
for root, dirs, files in os.walk(RUN_DIR):
    level = root.replace(RUN_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    sub_indent = ' ' * 2 * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path)
        size_str = f"{file_size / 1024:.1f}KB" if file_size > 1024 else f"{file_size}B"
        print(f'{sub_indent}{file} ({size_str})')

print(f"\n{'='*70}")
print("EXECUTION SUMMARY")
print(f"{'='*70}\n")

# Generate execution summary
summary_text = f"""
MULTICLASS ATTACK CLASSIFICATION RESULTS

Attack Classes: {len(class_labels)}
Attack Types: {', '.join(class_labels)}

Total Models Trained: {len(MODELS_TO_RUN)}
Models: {', '.join([m.upper() for m in MODELS_TO_RUN])}

Best Performing Model: {best_model_key.upper()}
Best F1-Score (Macro): {best_model_f1:.4f}

Results Location: {os.path.abspath(RUN_DIR)}

Generated Files:
- 01_metrics_summary_all_models.csv: Comprehensive metrics for all models
- 02_best_model_report.txt: Detailed analysis of the best model
- 03_models_comparison_metrics.png: Performance metrics comparison
- 04_f1_score_ranking.png: Model ranking by F1-score
- 05_metrics_heatmap.png: Metrics heatmap visualization

Model-Specific Outputs:
"""

for model_key in MODELS_TO_RUN:
    model_dir = os.path.join(RUN_DIR, model_key)
    if os.path.exists(model_dir):
        files = os.listdir(model_dir)
        summary_text += f"\n  {model_key.upper()}:\n"
        for file in sorted(files):
            summary_text += f"    - {file}\n"

summary_text += f"""
{'='*70}
"""

# Save summary to file
summary_path = os.path.join(RUN_DIR, "00_EXECUTION_SUMMARY.txt")
with open(summary_path, "w") as f:
    f.write(summary_text)

print(summary_text)
print(f"Summary saved to: {summary_path}")